In [1]:
%%javascript
//debugging to see if jupyter-klampt-widget is installed properly
console.log(require.s.contexts._.defined);
var kf = require('nbextensions/klampt/KlamptFrontend');
console.log(kf);

<IPython.core.display.Javascript object>

In [2]:
%load_ext wurlitzer
#^^^ used to capture C/C++ output to be displayed in the notebook

import time
from klampt import *
from IPython.display import clear_output
import ipyklampt

#get the HOME/Klampt base directory
import os
KLAMPT_DIR = os.environ['HOME'] + '/Klampt'


world = WorldModel()
world.loadFile(KLAMPT_DIR + "/data/athlete_plane.xml")
kvis = ipyklampt.KlamptWidget(world)
#If you'd like to hide the print output, uncomment this line
#clear_output()
display(kvis)

#Controls:
#left mouse click to rotate the view
#right click or ctrl+click to pan the view
#mouse wheel or shift+click to zoom the view

***  klampt.vis: using Qt as the visualization backend  ***


KlamptWidget(scene={u'object': {u'matrix': [1, 0, 0, 0, 0, 0, -1, 0, 0, 1, 0, 0, 0, 0, 0, 1], u'uuid': u'bg8b0…

In [3]:
q = world.robot(0).getConfig()
#getConfig() returns a copy of the robot's current configuration, so these changes
#aren't reflected in the WorldModel...
q[6] = 2
q[2] = 0.5
#until this call here
world.robot(0).setConfig(q)

#WAIT? why didn't the visualization change?
#... Because the WorldModel knows nothing about the visualization...

In [4]:
#... so we need to do this manually.
#Use a kvis.update() call to push the current state of the world to the visualization
kvis.update()

In [5]:
#do a little animation... note the kvis.update calls
q = world.robot(0).getConfig()
for i in range(20):
    q[2] = i*0.1
    world.robot(0).setConfig(q)
    kvis.update()
    time.sleep(0.1)

In [6]:
#can move the camera using get_camera() and set_camera()
cam = kvis.get_camera()
print cam
cam['target']['y'] += 0.2
kvis.set_camera(cam)

{u'near': 0.1, u'target': {u'y': 0, u'x': 0, u'z': 0}, u'far': 1000, u'position': {u'y': 3, u'x': 0, u'z': 6}, u'up': {u'y': 1, u'x': 0, u'z': 0}}


In [7]:
#add some "extras"

kvis.add_text(name="some_text",text="hello",x=10,y=10)
kvis.add_sphere(name="sphere1",x=1,y=0,z=0.5,r=0.3)
kvis.set_color("sphere1",1,0,0,0.25)
ghost = kvis.add_ghost()
kvis.set_color(ghost,0,1,0,0.5)
q = [0.0]*len(q)
kvis.set_ghost_config(q)

In [9]:
#items can be updated by calling add_ again.  Some attributes can be set to None to
#keep them at their previous values
kvis.add_text(name="some_text",text="hello modified",x=None,y=None)
kvis.add_sphere(name="sphere1",x=None,y=None,z=None,r=0.5)


In [11]:
#you can create multiple KlamptWidget objects
kvis2 = ipyklampt.KlamptWidget(world)
display(kvis2)
gh = kvis2.add_ghost('ghost')
kvis2.set_color(gh,0,1,0,0.5)

#and you can add editors
editor = ipyklampt.EditConfig(world.robot(0),kvis2,ghost='ghost',link_selector='dropdown')
#editor = ipyklampt.EditConfig(world.robot(0),kvis2,link_selector='slider')
#editor = ipyklampt.EditConfig(world.robot(0),kvis2,link_selector='all')
pt_editor = ipyklampt.EditPoint([1,0,0.5],klampt_widget=kvis2)
display(editor)
display(pt_editor)

KlamptWidget(scene={u'object': {u'matrix': [1, 0, 0, 0, 0, 0, -1, 0, 0, 1, 0, 0, 0, 0, 0, 1], u'uuid': u'ddg3g…

NameError: global name 'value' is not defined

In [ ]:
#you can even load multiple worlds
world2 = WorldModel()
world2.loadFile(KLAMPT_DIR+'/data/hubo_plane.xml')
kvis3 = ipyklampt.KlamptWidget(world2)

#here's how you show a playback widget 
playback_widget3 = jupyter.Playback(kvis3)
display(playback_widget3)
display(kvis3)

In [ ]:
#you can play around with these hooks to the playback widget
framerate = 30
movespeed = 1.0
robot = world2.robot(0)
q0 = robot.getConfig()
def advance():
    q = robot.getConfig()
    q[9] += movespeed/framerate
    robot.setConfig(q)
def reset():
    robot.setConfig(q0)
playback_widget3.advance = advance
playback_widget3.reset = reset
playback_widget3.framerate = framerate
playback_widget3.maxframes = 30